In [1]:
import argparse
import os
from epitome.models import *
from epitome.functions import *
from epitome.viz import *

from epitome.constants import *
import yaml
import subprocess
from timeit import default_timer as timer

/home/eecs/jahnavis/miniconda/envs/EpitomeNewEnv2/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


### Set Up

In [15]:
results_path = "results"
epitome_data_path = "data/epitome_data" 
feature_path = os.path.join(epitome_data_path, "feature_name")
TF = "JUND"
query_cell = 'K562' #'T47D'

In [16]:
# create user directories if they do not exist
epitome_results_dir = os.path.join(results_path, "epitome_results")
if not os.path.exists(epitome_results_dir):
    os.makedirs(epitome_results_dir)
    
tf_epitome_results_dir = os.path.join(epitome_results_dir, TF + "_results")
if not os.path.exists(tf_epitome_results_dir):
    os.makedirs(tf_epitome_results_dir)
    
model_dir = os.path.join(results_path, "epitome_models")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
tf_model_dir = os.path.join(model_dir, TF + "_models")
if not os.path.exists(tf_model_dir):
    os.makedirs(tf_model_dir)

In [4]:
epitome_data_path

'data/epitome_data'

### Load in Data for Epitome

In [28]:
train_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'train.npz')).toarray()
valid_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'valid.npz')).toarray()
test_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'test.npz')).toarray()
data = {Dataset.TRAIN: train_data, Dataset.VALID: valid_data, Dataset.TEST: test_data}
# all_data = np.concatenate((data[Dataset.TRAIN], data[Dataset.VALID], data[Dataset.TEST]), axis=1)

### VLP Model with Multiple TFs

In [ ]:
anchor_overlap_tfs = pd.read_csv(
    "/home/eecs/jahnavis/epitome_new/epitome-1/data/epitome_data/Anchor_Epitome_Overlap_TFs.csv")['TF'].tolist()
anchor_tfs = ["CTCF", "E2F1", "EGR1", "FOXA1", "FOXA2", "GABPA", "HNF4A", "JUND", 
              "MAX", "NANOG", "REST", "TAF1"]
# anchor_overlap_tfs = set(epitome_tfs).intersection(set(anchor_tfs))
# len(anchor_tfs), len(anchor_overlap_tfs), anchor_overlap_tfs

In [32]:
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = anchor_overlap_tfs,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2)
VLP(anchor_overlap_tfs,
    data = data,
    matrix = matrix,
    cellmap = cellmap,
    assaymap = assaymap)

In [37]:
TFs = ["CEBPB", "CHD2", "CTCF", "EP300", "GABPA", "JUND", "MAFK", "MAX", 
       "MYC", "NRF1", "RAD21", "REST", "RFX5", "SRF", "TAF1", "TBP", "USF2"]
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TFs,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2)
VLP(TFs,
    data = data,
    matrix = matrix,
    cellmap = cellmap,
    assaymap = assaymap)

using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'Ishikawa', 'IMR-90', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12892', 'GM12891', 'GM12878', 'A549'] as labels for mode Dataset.TRAIN
using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'Ishikawa', 'IMR-90', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12892', 'GM12891', 'GM12878', 'A549'] as labels for mode Dataset.TRAIN
using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'Ishikawa', 'IMR-90', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12892', 'GM12891', 'GM12878', 'A549'] as labels for mode Dataset.VALID


### VLP Model with Early Stopping (Single TF)

In [5]:
max_valid_iterations= 1000

model = VLP([TF], max_valid_records=max_valid_iterations)

start = timer()
iter_trained = model.train(2000) # train for 5000 iterations
end = timer()
train_time = end - start
print('epitome train: %f' % train_time)
model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + 
                          "_no_motif_early_stop_" + str(max_valid_iterations))
# model.save(model_path)

using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.TRAIN
using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.TRAIN
using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.VALID
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(40.450397, shape=(), dtype=float32)tf.Tensor(40.44542, shape=(), dtype=float32)tf.Tensor(0.004979495, shape=(), dtype=float32)
INFO:tensorflow:0 Validation Generator Time: 213.53789288923144 seconds
INFO:tensorflow:0 Validation:tf.Tensor(40.95179, shape=(), dtype=float32)
INFO:tensorflow:
INFO:tensorflow:1000 tf.Tensor(2.2502952, shape=(), dtype=float32)tf.Tensor(2.2452302, shape=(), dtype=float32)tf.Tensor(0.0050649494, shape=(), dtype=float32)
INFO:tensorflow:1000 Validation Generator Time: 209.84591072797775 seconds
INFO:tensorflow:1000 Validation:tf.Tensor(5.357682, shape=(), dtype=float32)
INFO:tensorflow:
epitome train: 958.689393


In [ ]:
model_results = model.test(10000, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

In [ ]:
eval_results_df = pd.DataFrame(columns=['transcription_factor', 'query_cell', 'auROC', 'auPRC'])
eval_results_df = eval_results_df.append({ 
   'transcription_factor' : TF,
   'query_cell' : query_cell,
   'auROC' : model_results['auROC'],
   'auPRC' : model_results['auPRC'],
   'iterations_trained' : iter_trained,
   'train_time': train_time}, 
    ignore_index=True)

eval_results_dir = os.path.join(tf_epitome_results_dir, query_cell + "_" + TF + 
                           '_no_motif_early_stop_'+ str(max_valid_iterations) + 
                           '.csv')
eval_results_df.to_csv(eval_results_dir, sep="\t")

### VLP Model Without Early Stopping

In [ ]:
# matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
#                                                          eligible_assays = None,
#                                                          eligible_cells = None, 
#                                                          min_cells_per_assay = 2, 
#                                                          min_assays_per_cell= 2) #10)

In [6]:
TF = "JUND"
model = VLP([TF])

start = timer()
iter_trained = model.train(500) # train for 5000 iterations
end = timer()
train_time = end - start
print('epitome train: %f' % train_time)

model_results = model.test(10000, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

using ['MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'H1', 'A549'] as labels for mode Dataset.TRAIN
using ['MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'H1', 'A549'] as labels for mode Dataset.TRAIN
using ['MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'H1', 'A549'] as labels for mode Dataset.VALID
Instructions for updating:
Please use `layer.add_weight` method instead.
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(43.22754, shape=(), dtype=float32)tf.Tensor(43.217567, shape=(), dtype=float32)tf.Tensor(0.009973269, shape=(), dtype=float32)
INFO:tensorflow:
epitome train: 136.933354


In [7]:
model_results = model.test(10000, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

157it [00:35,  4.44it/s]

INFO:tensorflow:macro auROC:     0.9811924769907964
INFO:tensorflow:auPRC:     0.020643831303481
INFO:tensorflow:GINI:     0.9623849539815926
Model auROC: 0.9811924769907964. Model auPRC: 0.020643831303481.


In [ ]:
eval_results_df = pd.DataFrame(columns=['transcription_factor', 'query_cell', 'auROC', 'auPRC'])
eval_results_df = eval_results_df.append({ 
   'transcription_factor' : TF,
   'query_cell' : query_cell,
   'auROC' : model_results['auROC'],
   'auPRC' : model_results['auPRC'],
   'iterations_trained' : iter_trained,
   'train_time': train_time}, 
    ignore_index=True)

eval_results_df.to_csv(os.path.join(tf_epitome_results_dir,
                                    query_cell + "_" + TF + '_no_motif' + '.csv'), sep="\t")

In [11]:
model_results['preds_mean'].shape

(10000, 1)

In [36]:
TF = "JUND"

matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TF,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2)
    
model2 = VLP([TF],
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap)

start = timer()
iter_trained = model2.train(500) # train for 5000 iterations
end = timer()
train_time = end - start
print('epitome train: %f' % train_time)

model2_results = model2.test(10000, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

In [30]:
model2_results['preds_mean'].shape

(10000, 2)

In [35]:
model2_results

{'preds_mean': array([[0.12285104, 0.08643743],
        [0.12394705, 0.0884769 ],
        [0.13021478, 0.08571389],
        ...,
        [0.2193261 , 0.13365811],
        [0.15296759, 0.11804638],
        [0.15474007, 0.11731315]], dtype=float32),
 'preds_std': array([[0.01769408, 0.0136858 ],
        [0.02070205, 0.01198193],
        [0.02263527, 0.01317834],
        ...,
        [0.02264554, 0.0143347 ],
        [0.01964175, 0.01462081],
        [0.01843329, 0.01442528]], dtype=float32),
 'truth': array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=float32),
 'weights': array([[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 1.],
        [1., 0.],
        [1., 1.]], dtype=float32),
 'assay_dict': {'JUND': {'AUC': 0.9554295876403246,
   'auPRC': 0.02699860869218639,
   'GINI': 0.8811707786555936},
  'JUN': {'AUC': 0.9973328888148025,
   'auPRC': 0.058823529411764705,
   'GINI': 0.46545285

In [25]:
JUND_results = os.path.join(tf_epitome_results_dir, "K562_JUND_motif_anchor.npz")
JUND_preds = np.load(JUND_results)

In [26]:
JUND_preds['pred'].shape

(10000, 2)